<a href="https://colab.research.google.com/github/TiagolGoulart/GeoPython/blob/main/clip_raster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio

In [3]:
# Import the libraries
import geopandas as gpd
import rasterio.mask
from rasterio.plot import show
import numpy as np

In [ ]:
# Read the shapefile that will serve as a clipping mask
shape_path = "C:\Users\Shapefiles\mask.shp"
shape = gpd.read_file(shape_path)

In [ ]:
# Define the function that will transform the shapefile geometries to json in the format that the rasterio understands for the clipping
def getFeatures(gdf):
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
# Run the function defined above
coords = getFeatures(shape)

In [ ]:
# Use rasterio to open the image
raster_path = "C:\Users\Rasters\raster.tif"

In [ ]:
with rasterio.open(raster_path) as src:
    # Use the mask function to crop the image with the coordinates extracted from the shapefile (coords)
    out_image, out_transform = rasterio.mask.mask(src, coords, crop=True)
    # Get the original raster metadata
    out_meta = src.meta
    b2=src.read(1)
    affine=src.transform
# Update the original raster metadata with new dimensions and transformation to geographic coordinates
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

# Save the clipping in a new raster
with rasterio.open("raster.masked.tif", "w", **out_meta) as dest:
    dest.write(out_image)